In [132]:
import xgboost
import joblib
import numpy as np
import pandas as pd

#### Load model, feature names, testing data and counterfactuals from testing set

In [133]:
#model = joblib.load('files/model_asap1_776')
#model = joblib.load('files/model_asap1_776_counter_both_ori')
model = joblib.load('files/model_asap1_776_counter_both_ori_adjusted')

feature_names = joblib.load('files/feature_names_776')

data_test_ori = pd.read_csv('ASAP1 Test Set.tsv', sep='\t')

feat_test_ori = joblib.load('files/asap1_features_test_776')

feats_counter_test_776 = joblib.load('files/feats_counter_test_correct_776')
np.array(feats_counter_test_776).shape

(100, 31, 1, 776)

In [134]:
feat_test_ori.shape

(357, 776)

In [135]:
# feats_counter_test_776 = np.delete(feats_counter_test, [5,6],axis=3)
# feats_counter_test_776.shape

In [136]:
# joblib.dump(feats_counter_test_776, 'feats_counter_test_776')

#### Predict the scores of the original essays

In [137]:
scores_ori = []
row_ctr = 0

for feat in feat_test_ori[:100]:
    dmat = xgboost.DMatrix(feat.reshape(1,-1), feature_names=feature_names)
    score = model.predict(dmat)
    scores_ori.append(score[0])
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [138]:
#joblib.dump(scores_ori, "scores_ori_test_776")
# scores_ori = joblib.load("scores_ori_test_776")

#### Predict the scores of the counterfactuals

In [139]:
scores_counter = []

row_ctr = 0

for row in feats_counter_test_776:
    row_score = []
    for essay_feat in row:
        essay_feat = xgboost.DMatrix(essay_feat, feature_names=feature_names)
        score = model.predict(essay_feat)
        row_score.append(score[0])
    scores_counter.append(row_score)
    row_ctr = row_ctr + 1
    print("Finished row number : ", row_ctr)

print("row number: ", row_ctr)

Finished row number :  1
Finished row number :  2
Finished row number :  3
Finished row number :  4
Finished row number :  5
Finished row number :  6
Finished row number :  7
Finished row number :  8
Finished row number :  9
Finished row number :  10
Finished row number :  11
Finished row number :  12
Finished row number :  13
Finished row number :  14
Finished row number :  15
Finished row number :  16
Finished row number :  17
Finished row number :  18
Finished row number :  19
Finished row number :  20
Finished row number :  21
Finished row number :  22
Finished row number :  23
Finished row number :  24
Finished row number :  25
Finished row number :  26
Finished row number :  27
Finished row number :  28
Finished row number :  29
Finished row number :  30
Finished row number :  31
Finished row number :  32
Finished row number :  33
Finished row number :  34
Finished row number :  35
Finished row number :  36
Finished row number :  37
Finished row number :  38
Finished row number :

In [141]:
np.array(scores_counter).shape

(100, 31)

In [142]:
#joblib.dump(scores_counter, "scores_counterfactual_test_776")
#scores_counter = joblib.load('scores_counterfactual_test')

#### Compare the scores Original vs Counterfactual

In [143]:
comparison = []

ctr = 1

for idx, row_scores in enumerate(scores_counter):
    ori_score = scores_ori[idx]
    compare_row = []
    for score in row_scores:

        if ctr <= 10:
            print(ctr)
            print("ori: ", ori_score)
            print("counter: ", score)
        
        score_diff = abs(score - ori_score)
        if score >= ori_score:
            compare_row.append((True, score_diff, score))
        else:
            compare_row.append((False, score_diff, score))
        ctr = ctr + 1
        
    comparison.append(compare_row)

1
ori:  6.164053
counter:  10.097233
2
ori:  6.164053
counter:  10.121732
3
ori:  6.164053
counter:  9.898743
4
ori:  6.164053
counter:  9.7192955
5
ori:  6.164053
counter:  10.17938
6
ori:  6.164053
counter:  10.479513
7
ori:  6.164053
counter:  10.251809
8
ori:  6.164053
counter:  10.081911
9
ori:  6.164053
counter:  10.316269
10
ori:  6.164053
counter:  10.221785


In [144]:
comparison

[[(True, 3.9331799, 10.097233),
  (True, 3.9576788, 10.121732),
  (True, 3.7346897, 9.898743),
  (True, 3.5552425, 9.7192955),
  (True, 4.0153275, 10.17938),
  (True, 4.31546, 10.479513),
  (True, 4.087756, 10.251809),
  (True, 3.9178581, 10.081911),
  (True, 4.152216, 10.316269),
  (True, 4.0577316, 10.221785),
  (True, 3.9576788, 10.121732),
  (True, 4.5287323, 10.692785),
  (True, 3.7615633, 9.925616),
  (True, 4.2475014, 10.411554),
  (True, 4.063712, 10.227765),
  (True, 4.5005894, 10.664642),
  (True, 4.441452, 10.605505),
  (True, 4.625411, 10.789464),
  (True, 4.1396275, 10.30368),
  (True, 4.5453014, 10.709354),
  (True, 4.152216, 10.316269),
  (True, 4.0577316, 10.221785),
  (True, 4.516547, 10.6806),
  (True, 4.600751, 10.764804),
  (True, 4.1396275, 10.30368),
  (True, 4.5005894, 10.664642),
  (True, 4.6009855, 10.7650385),
  (True, 4.7820005, 10.9460535),
  (True, 4.3588495, 10.5229025),
  (True, 4.5005894, 10.664642),
  (True, 4.6009855, 10.7650385)],
 [(True, 3.8856993, 

#### Count the number of False and mean of score difference for the 'False' predictions

In [145]:
# Initialize counters and lists to hold False values' second column values
false_count = 0
false_values = []
new_label = []
true_count = 0

# Iterate through each sublist
for idx, sublist in enumerate(comparison):
    ori = scores_ori[idx]
    row_label = []
    for value in sublist:

        formula = ori + value[1]
        
        if value[0] == False:  # If the first element is False
            false_count += 1
            false_values.append(value[1])  # Collect the second element (the difference)
            row_label.append(formula)
        else:
            true_count += 1
            row_label.append(value[2])
    new_label.append(row_label)
    
# Calculate the mean of the differences for False values only
if false_count > 0:
    mean_difference = np.mean(false_values)
else:
    mean_difference = 0  # Handle case where there are no False values

print(f"Count of False values: {false_count}")
print(f"Count of True values: {true_count}")
print(f"Mean of the differences for False values: {mean_difference}")


Count of False values: 844
Count of True values: 2256
Mean of the differences for False values: 0.8406384587287903


In [146]:
np.array(comparison).shape

(100, 31, 3)

In [147]:
acc = (true_count / 3100) * 100
acc

72.7741935483871

In [148]:
np.array(new_label).shape

(100, 31)

In [37]:
#joblib.dump(new_label, 'new_label_test_776')